## Máster en Ingeniería informática: Datos, Cloud y Gestión TI

# Análisis de información no estructurada

## Generador de letras de canciones

#### Realizado por: Arturo Pérez Sánchez y Jacinto Ruiz Díaz

---

El objetivo de este proyecto es el de implementar un sistema que permita generar de manera automática letras de canciones imitando el estilo de un artista en particular, realizando previamente una lectura de la letra de sus canciones.

## Recogida de datos: web scraping

El primer paso a realizar es la extracción de las letras de canciones para su posterior análisis. En nuestro caso, para obtener las letras de las canciones hemos optado por realizar <a href="https://es.wikipedia.org/wiki/Web_scraping">web scrapping</a> de la página <a href="https://www.azlyrics.com/">azlyrics</a>, la cual almacena letras de una amplia variedad de autores.

Para realizar la extracción de las letras utilizaremos la libreria <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">BeautifulSoup</a>, al ser una de las librerías de Python más completas e intuitivas para realizar este tipo de tareas.

Comenzaremos importándola:

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json 
import io
import re

Una vez importada la librería definiremos algunos Hiperparámetros:
<ul>
    <li><strong>group:</strong> Nombre del grupo musical cuyo estilo deseamos imitar.</li>
    <li><strong>albums:</strong> Lista de álbumes del grupo que vamos a analizar (si se deja vacío se leerán todos).</li>
    <li><strong>wait_time:</strong> Tiempo en segundos que transcurrirá entre una petición y otra.</li>
</ul>

In [101]:
group = 'Lady Gaga'
albums = ['Red And Blue', 'The Fame', 'The Fame Monster', 'Born This Way', 'A Very Gaga Holiday', 'Artpop', 'Joanne', 'Chromatica']
wait_time = 3

Lo primero que tenemos que realizar es una conversión del nombre del grupo al nombre utilizado en la página (todo en minúsculas y sin espacios):

In [102]:
group = group.lower().replace(" ", "")

Ahora procedemos a estudiar cómo se estructuran las URLs del sitio web al que vamos a hacer scrapping.

Concretamente, estamos interesados en __dos__ vistas del sitio:

### Lista de canciones separadas por álbumes
La URL de esta página se estructura de la forma:
<h3><center>https://www.azlyrics.com/<font color="red">[initial]</font>/<font color="red">[group]</font>.html </center></h3>
Donde <i>[initial]</i> corresponde a la primera letra del grupo* y <i>[group]</i> corresponde al nombre del grupo. En el ejemplo del grupo Lady Gaga, se vería de la siguiente forma:

<img src="images/gaga_album_example.png" width=600>
<br>

*_Si el nombre del grupo comienza por un carácter especial la variable [initial] tomará el valor '19'_

### Letra de cada canción
En este caso, notaremos que la ruta es la misma pero cambiando [initial] por 'lyrics' y añadiendo el nombre de la canción justo después del nombre del grupo:

<h3><center> https://www.azlyrics.com/lyrics/<font color="red">[group]</font>/<font color="red">[song]</font>.html </center></h3>

El resultado para el ejemplo de la canción "Poker Face" de Lady Gaga es el siguiente:

<img src="images/gaga_lyrics2.png" width=600>

Con esto, ya podemos estructurar la ruta de álbumes a partir de los hiperparámetros que habíamos establecido:

In [103]:
#Inicializamos la variable initial como el primer caracter del grupo o 19 si no es una letra
initial = group[0] if group[0].isalpha() else '19'

#Prefijo del sitio web
prefix = 'https://www.azlyrics.com'
sufix = '.html'

#Concatenamos las variables para formar la URL completa
albumsUrl = prefix + '/' + initial + '/' + group

Con la URL de los álbumes ya creados, podemos utilizar las librerías anteriormente mencionadas para hacer la llamada a la página y obtener la lista de álbumes y los títulos de las canciones que lo componen:

In [104]:
# Las claves serán los álbumes y los valores un array con las canciones. Las canciones a su vez serán un array de tamaño 3:
# El primer valor es el titulo, el segundo el enlace y el tercero la letra de la canción
albumsDict = {}

# Hacemos la llamada para obtener la lista de álbumes
r = requests.get(albumsUrl + sufix)
r.encoding = 'utf-8' #Codificamos el request a utf-8

soup = BeautifulSoup(r.text, 'lxml')

res = soup.findAll('div', class_=['album', 'listalbum-item'])

# Recorremos la lista de álbumes y para cada uno de ellos guardamos en el diccionario
# el título de cada canción y la ruta a la letra de dicha canción
currentAlbum = ''
for div in res:
    if(div.get('class')[0] == 'album'):
        # Este div corresponde a un álbum, por lo que añadimos una nueva entrada al diccionario
        currentAlbum = div.find('b').contents[0].replace('"', '')
        albumsDict[currentAlbum] = []
    else:
        # Este div corresponde a una canción por lo que la añadimos a los valores del álbum
        # Hay veces que referencia a una ruta completa, al pertenecer a otro artista, por lo que hay que tenerlo en cuenta
        if div.contents[0]['href'][2:].startswith("tps://www.azlyrics.com"):
            albumsDict[currentAlbum].append([div.contents[0].contents[0], div.contents[0]['href']])
        else:
            albumsDict[currentAlbum].append([div.contents[0].contents[0], 'https://www.azlyrics.com' + div.contents[0]['href'][2:]])

Ahora tenemos un diccionario con todos los álbumes, y para cada álbum tenemos todas las canciones que lo componen con sus respectivos enlaces a la letra. Por tanto, solo nos faltaría llamar a cada uno de estos enlaces para obtener la letra de cada canción; sin embargo, para reducir el número de llamadas al sitio web, primero realizaremos una limpieza de los álbumes para quedarnos solo con aquellos que habíamos indicado que nos interesaban:

In [57]:
# Limpieza de álbumes
# Del diccionario que hemos creado nos quedamos solo con los álbumes 
# que aparezcan en el array de álbumes que habíamos especificado inicialmente o con todos si no se ha especificado
if(albums):
    albumsDict = {a: albumsDict[a] for a in albums if a in albumsDict}

Ahora sí que tenemos todo listo para realizar la lectura de las canciones. Cabe destacar que entre cada llamada dejaremos un espacio de tiempo (indicado en la variable '_wait_time_') para evitar que el sitio web nos bloquee la dirección IP. Para conseguirlo haremos uso de la librería **time**.

In [88]:
#Creamos un diccionario donde las claves serán las canciones y el valor la letra de dicha canción
lyricsDict = {}

print('Leyendo ' + str(len(albumsDict)) + ' álbum(es)')

#Recorremos la lista de álbumes y para cada álbum recorremos la lista de canciones
for album in albumsDict:
    print ('Reading album: ', album)
    for song in albumsDict[album]:
        print ('    Reading song: ', song[0])
        
        # Esperamos unos segundos antes de realizar la llamada
        time.sleep(wait_time)
        
        r = requests.get(song[1])
        r.encoding = 'utf-8' #Codificamos el request a utf-8
        
        soup = BeautifulSoup(r.text, 'lxml')
        
        # Como el div que contiene la letra no tiene ninguna clase ni identificador lo obtendremos a partir del div padre
        column = soup.find('div', class_=['col-xs-12 col-lg-8 text-center'])
        
        #El div con la letra de la canción siempre estará en 5º lugar, después de los divs del título y de las redes sociales
        raw_lyrics = column.findAll('div')[5].text
        raw_lyrics = re.sub("[\(\[].*?[\)\]]", "", raw_lyrics) #Quitamos el contenido entre paréntesis y corchetes
        lyricsDict[song[0]] = raw_lyrics

Leyendo 0 álbum(es)


Por último, tenemos que fusionar los arrays albumsDict y lyricsDict:

In [22]:
# Recorremos los álbumes
for album in albumsDict:
    # Recorremos las canciones de cada álbum
    for song in albumsDict[album]:
        #A cada canción le añadimos la letra correspondiente
        song.append(lyricsDict[song[0]])

Ahora ya tenemos todas las canciones tal y como queríamos, por lo que podemos guardarlas en un json para su posterior análisis:

In [30]:
# El json se guardará en la carpeta data
with open('data/' + group + "-lyrics.json", "w", encoding='utf-8') as outfile: 
    json.dump(albumsDict, outfile)

Vamos a abrir el fichero CSV generado para ver qué forma tiene:

In [31]:
with open('data/julioiglesias-lyrics.json') as f:
     data = json.load(f)

print(json.dumps(data, indent=4))

{
    "Yo Canto": [
        [
            "La Vida Sigue Igual",
            "https://www.azlyrics.com/lyrics/julioiglesias/lavidasigueigual.html",
            "\n\r\nUnos que nacen, otros morir\u00e1n;\nUnos que r\u00eden, otros llorar\u00e1n.\nAguas sin cauce, r\u00edos sin mar,\nPenas y glorias, guerras y paz.\n\nSiempre hay\nPor que vivir,\nPor que luchar.\n\nSiempre hay\nPor quien sufrir\nY a quien amar.\n\nAl final\nLas obras quedan, las gentes se van.\nOtros que vienen las continuar\u00e1n.\nLa vida sigue igual!\n\nPocos amigos que son de verdad;\nCuantos te alagan si triunfando estas;\nY si fracasas, bien comprender\u00e1s:\nLos buenos quedan, los dem\u00e1s se van.\n\nSiempre hay\nPor que vivir,\nPor que luchar.\n\nSiempre hay\nPor quien sufrir\nY a quien amar.\n\nAl final\nLas obras quedan, las gentes se van.\nOtros que vienen las continuar\u00e1n.\nLa vida sigue igual!\n\nAl final\nLas obras quedan, las gentes se van.\nOtros que vienen las continuar\u00e1n.\nLa vida sigue ig

Sigue la forma: <br><br> <center><font size="3"> { nombreAlbum: [ [tituloCancion, urlCancion, letraCancion] ]} </font></center>

Para mostrar un álbum en concreto:

In [13]:
#Álbum Red And Blue
data['Red And Blue']

[['Something Crazy',
  'https://www.azlyrics.com/lyrics/ladygaga/somethingcrazy.html',
  "\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy t

Para mostrar una canción de un álbum en particular:

In [20]:
#Álbum Red And Blue, canción Something Crazy
data['Red And Blue'][0]

['Something Crazy',
 'https://www.azlyrics.com/lyrics/ladygaga/somethingcrazy.html',
 "\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to m

Para quedarnos con la letra de dicha canción:

In [21]:
#Álbum Red And Blue, letra de la canción Something Crazy
data['Red And Blue'][0][2]

"\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen 

Para ver los nombres de los álbumes almacenados:

In [32]:
for i in data:
    print(i)

Red And Blue
The Fame
The Fame Monster
Born This Way
A Very Gaga Holiday
Artpop
Joanne
Chromatica


Para ver los títulos de las canciones de un álbum en particular:

In [28]:
for i in data['Red And Blue']:
    print(i[0])

Something Crazy
Wish You Were Here
No Floods
Red And Blue
Words


---

### Obteniendo todos los artistas disponibles de azlyrics

In [65]:
# Cada uno de los índices corresponde a una página del tipo https://www.azlyrics.com/{index}.html, que contiene los 
# artistas que empiezan por dicho carácter
indexes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
           'w', 'x', 'y', 'z', '19']
baseUrl = "https://www.azlyrics.com/"
sufix = ".html"

wait_time = 5

for i in indexes:
    # Esperamos unos segundos antes de realizar la llamada
    time.sleep(wait_time)
    
    group = baseUrl + i + sufix
    
    r = requests.get(group)
    soup = BeautifulSoup(r.text, 'lxml')

    divs = soup.find_all('div', class_=['col-sm-6 text-center artist-col'])
    
    for d in divs:
        links.extend(d.find_all('a'))
    
    for a in links:
        print(a.text + " " + baseUrl + a['href'])
    break

A1 https://www.azlyrics.com/a/a1.html
A1 https://www.azlyrics.com/f/floyda1bentley.html
A1 x J1 https://www.azlyrics.com/a/a1xj1.html
A https://www.azlyrics.com/a/a.html
A2H https://www.azlyrics.com/a/a2h.html
A92 https://www.azlyrics.com/a/a92.html
Aalegra, Snoh https://www.azlyrics.com/s/snohaalegra.html
Aaliyah https://www.azlyrics.com/a/aaliyah.html
Aalto, Saara https://www.azlyrics.com/s/saaraaalto.html
Aaradhna https://www.azlyrics.com/a/aaradhna.html
Aaron Carpenter https://www.azlyrics.com/a/aaroncarpenter.html
Aaron Carter https://www.azlyrics.com/c/carter.html
Aaron Cole https://www.azlyrics.com/a/aaroncole.html
Aaron Doh https://www.azlyrics.com/a/aarondoh.html
Aaron Fresh https://www.azlyrics.com/a/aaronfresh.html
Aaron Goodvin https://www.azlyrics.com/a/aarongoodvin.html
Aaron Hall https://www.azlyrics.com/a/aaronhall.html
Aaron Lewis https://www.azlyrics.com/a/aaronlewis.html
Aaron Lines https://www.azlyrics.com/l/lines.html
Aaron May https://www.azlyrics.com/a/aaronmay.h

Allen, Kris https://www.azlyrics.com/k/krisallen.html
Allen - Lande https://www.azlyrics.com/a/allenlande.html
Allen, Lily https://www.azlyrics.com/l/lilyallen.html
Allen Stone https://www.azlyrics.com/a/allenstone.html
Alley, Candice https://www.azlyrics.com/c/candice.html
All Get Out https://www.azlyrics.com/a/allgetout.html
All Good Things https://www.azlyrics.com/a/allgoodthings.html
Allie X https://www.azlyrics.com/a/alliex.html
Alligatoah https://www.azlyrics.com/a/alligatoah.html
Alli Simpson https://www.azlyrics.com/a/allisimpson.html
Allison Iraheta https://www.azlyrics.com/a/allisoniraheta.html
Allister https://www.azlyrics.com/a/allister.html
Allj (Ð­Ð»Ð´Ð¶ÐµÐ¹) https://www.azlyrics.com/a/allj.html
Allman Brothers Band, The https://www.azlyrics.com/a/allmanbrothersband.html
Allman Brown https://www.azlyrics.com/a/allmanbrown.html
Allman, Gregg https://www.azlyrics.com/g/greggallman.html
ALLMO$T https://www.azlyrics.com/a/allmot.html
All Saints https://www.azlyrics.com/a/alls

Amaru, Benjamin https://www.azlyrics.com/b/benjaminamaru.html
Amazons, The https://www.azlyrics.com/a/amazons.html
Ambar Lucid https://www.azlyrics.com/a/ambarlucid.html
Amberian Dawn https://www.azlyrics.com/a/amberiandawn.html
Amber Mark https://www.azlyrics.com/a/ambermark.html
Amber Pacific https://www.azlyrics.com/a/amberpacific.html
Amber Riley https://www.azlyrics.com/a/amberriley.html
Amber Rose https://www.azlyrics.com/a/amberrose.html
Amber Run https://www.azlyrics.com/a/amberrun.html
Ambjaay https://www.azlyrics.com/a/ambjaay.html
Ambrosia https://www.azlyrics.com/a/ambrosia.html
Ambrosius, Marsha https://www.azlyrics.com/m/marshaambrosius.html
Ambush Buzzworl https://www.azlyrics.com/a/ambushbuzzworl.html
Ameer Vann https://www.azlyrics.com/a/ameervann.html
Amel Bent https://www.azlyrics.com/a/amelbent.html
Amelia Lily https://www.azlyrics.com/a/amelialily.html
Amelia Monet https://www.azlyrics.com/a/ameliamonet.html
Amel Larrieux https://www.azlyrics.com/a/amellarrieux.htm

Answer, The https://www.azlyrics.com/a/answer.html
Ant Beale https://www.azlyrics.com/a/antbeale.html
Ant Clemons https://www.azlyrics.com/a/antclemons.html
Anth https://www.azlyrics.com/a/anth.html
Anthem Lights https://www.azlyrics.com/a/anthemlights.html
Anthony Amorim https://www.azlyrics.com/a/anthonyamorim.html
Anthony Brown & group therAPy https://www.azlyrics.com/a/anthonybrown.html
Anthony Callea https://www.azlyrics.com/a/anthonycallea.html
Anthony David https://www.azlyrics.com/a/anthonydavid.html
Anthony Evans https://www.azlyrics.com/a/anthonyevans.html
Anthony Green https://www.azlyrics.com/a/anthonygreen.html
Anthony Hamilton https://www.azlyrics.com/h/hamilton.html
Anthony Lewis https://www.azlyrics.com/a/anthonylewis.html
Anthony, Marc https://www.azlyrics.com/a/anthony.html
Anthony Ramos https://www.azlyrics.com/a/anthonyramos.html
Anthony Russo https://www.azlyrics.com/a/anthonyrusso.html
Anthrax https://www.azlyrics.com/a/anthrax.html
Anti-Flag https://www.azlyrics.

### Generando el fichero de letras

Ahora que tenemos de forma estructurada toda la información necesaria, recorreremos el json para almacenar en un fichero txt todas las letras de las canciones recogidas, que nos servirá para realizar el posterior entrenamiento del modelo.

In [32]:
file = io.open("song_lyrics.txt", "w", encoding="utf-8") 

n_songs = 0

for album in data:
    for song in data[album]:
        file.write(song[2]) #El índice 2 contiene las letras
        n_songs += 1

print("Número total de canciones almacenadas: {}.\n".format(n_songs))

file.close()

print("song_lyrics.txt creado satisfactoriamente.")


Número total de canciones almacenadas: 53.

song_lyrics.txt creado satisfactoriamente.


Usaremos el fichero song_lyrics.txt para entrenar el modelo.

----

## Entrenamiento del modelo

In [33]:
#Importamos las librerías necesarias
# pip3 install aitextgen
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

In [106]:
# Indicamos el fichero a utilizar para entrenar el modelo
file_name = "song_lyrics.txt"

In [107]:
# Train a custom BPE Tokenizer on the downloaded text
# This will save one file: `aitextgen.tokenizer.json`, which contains the
# information needed to rebuild the tokenizer.
train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"

In [108]:
# GPT2ConfigCPU is a mini variant of GPT-2 optimized for CPU-training
# e.g. the # of input tokens here is 64 vs. 1024 for base GPT-2.
config = GPT2ConfigCPU()

In [109]:
# Instantiate aitextgen using the created tokenizer and config.
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)

# You can build datasets for training by creating TokenDatasets,
# which automatically processes the dataset with the appropriate size.
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)

In [110]:
# Train the model! It will save pytorch_model.bin periodically and after completion to the `trained_model` folder.
# On a 2020 8-core iMac, this took ~25 minutes to run.
ai.train(data, batch_size=16, num_steps=10000)

pytorch_model.bin already exists in /trained_model and will be overwritten!
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
 leave
Where are our leave
In the girl, yeah, yeah
I can be man
I'm gon' be man cure
' hure
I wanna be man, take manicured
You wanna be man cure at you feel you

Ma ma manic
2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.
 reven lives me if caught
But it's the slaps
I can't retrol
But it's just a man,
Now it's get it's a fire
Hearday


All of the thing that you don't play with me

3,000 steps reached: saving model to /trained_model
3,000 steps reached: generating sample texts.
, I'd makes meet
And a game!
Hurt forget you'd do you more than doe
ARTPOP, you love a game?
(Oh-oh-oh-oh-oh-oh-oh-oh-oh-oh-oh-oh-oh-
4,000 steps reached: saving model to /trained_model
4,000 steps reached: generating sample texts.
n that I'm act like a knirader
Scessately 
Summer boy

I've got a little sun goes down
You've got to meummer boy

You can't me go for it 


In [113]:
ai.generate(n=3, prompt="God", max_length=100, temperature=1.3)

Goderstasy aday, I am beyond
My phhere before hear my spring forgets.
That's love is the world?
And a mevastic dance.
You've got your longerian a hooker

Fian hookers
Goddess of love
In love, love, stupid love

Rot, blow is that way
I want love

We can be shy sy
The fight've decked
But I like you just like least imag?
Who
Goder innomourty)
Twlight have got a forever even threpice love you pleaseures
Baby don't jew me, eh

Right
But I looked at anything

It's hard only friend
ë
